In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
datos = pd.read_csv('datos_procesamiento.csv')
display(datos.head())
print(datos.shape)

,col_inexistente1,col2,col3,col_outliers,col_outliers2,col_categorica,col_ordinal,col_texto
0,59.0,52.0,2.232832,-50,0.771666,ratón,muy bien,Tenía en su casa una ama que pasaba de los cua...
1,31.0,74.0,0.906147,-5,1.068558,elefante,regular,"El resto della concluían sayo de velarte, calz..."
2,81.0,28.0,0.626750,-32,0.846396,ratón,muy mal,"El resto della concluían sayo de velarte, calz..."
3,34.0,16.0,0.816738,-84,0.637381,gato,mal,"Una olla de algo más vaca que carnero, salpicó..."
4,32.0,28.0,0.571131,65,4.540614,gato,bien,Tenía en su casa una ama que pasaba de los cua...


(1000, 8)


In [ ]:
datos.dtypes

,0
col_inexistente1,float64
col2,float64
col3,float64
col_outliers,int64
col_outliers2,float64
col_categorica,object
col_ordinal,object
col_texto,object


In [ ]:
var_numericas_df = datos.select_dtypes(include=[int, float])
var_numericas_df.dtypes

,0
col_inexistente1,float64
col2,float64
col3,float64
col_outliers,int64
col_outliers2,float64


In [ ]:
var_numericas_df[var_numericas_df.isnull().any(axis=1)].shape

(96, 5)

In [ ]:
from sklearn.impute import SimpleImputer
imputador = SimpleImputer(missing_values=np.nan, copy=False, strategy="mean")

In [ ]:
var_numericas_imputadas = imputador.fit_transform(var_numericas_df)

In [ ]:
var_numericas_imputadas

array([[ 59.        ,  52.        ,   2.23283208, -50.        ,
          0.77166646],
       [ 31.        ,  74.        ,   0.90614714,  -5.        ,
          1.06855838],
       [ 81.        ,  28.        ,   0.62675042, -32.        ,
          0.84639576],
       ...,
       [ 19.        ,  53.        ,   0.73723413,  73.        ,
          1.34525201],
       [ 88.        ,  94.        ,   0.76008706,  68.        ,
          1.3692463 ],
       [ 94.        ,  56.        ,   1.2299403 ,  61.        ,
          0.94395714]])

In [ ]:
var_numericas_imputadas.shape

(1000, 5)

In [ ]:
var_numericas_imputadas_df =pd.DataFrame(var_numericas_imputadas, columns=var_numericas_df.columns, index=var_numericas_df.index)
var_numericas_imputadas_df.head()

,col_inexistente1,col2,col3,col_outliers,col_outliers2
0,59.0,52.0,2.232832,-50.0,0.771666
1,31.0,74.0,0.906147,-5.0,1.068558
2,81.0,28.0,0.626750,-32.0,0.846396
3,34.0,16.0,0.816738,-84.0,0.637381
4,32.0,28.0,0.571131,65.0,4.540614


In [ ]:
var_numericas_imputadas_df[var_numericas_imputadas_df.isnull().any(axis=1)].shape

(0, 5)

Estandarize.
El proceso de Estandarización es un proceso requerido por una gran cantidad de modelos en Scikit-learn. El objetivo es obtener una variable con media 0 y desviación estándar 1.

In [ ]:
var_numericas_df.mean()

,0
col_inexistente1,48.382743
col2,49.660000
col3,1.466095
col_outliers,4.253000
col_outliers2,131.193340


In [ ]:
var_numericas_df.std()

,0
col_inexistente1,27.987174
col2,28.272668
col3,1.732358
col_outliers,78.145901
col_outliers2,3401.164776


In [ ]:
from sklearn import preprocessing
escalador = preprocessing.StandardScaler()
var_numericas_imputadas_escalado_standard = escalador.fit_transform(var_numericas_imputadas)

In [ ]:
escalador.mean_

array([ 48.38274336,  49.66      ,   1.46609489,   4.253     ,
       131.19333968])

In [ ]:
var_numericas_imputadas_escalado_standard.mean(axis=0)

array([-5.86197757e-17,  1.26121336e-16, -3.90798505e-17,  3.55271368e-18,
       -3.55271368e-18])

In [ ]:
var_numericas_imputadas_escalado_standard.std(axis=0)

array([1., 1., 1., 1., 1.])

In [ ]:
var_numericas_imputadas_escalado_standard[0]

array([ 0.39921733,  0.08280686,  0.44281884, -0.69460006, -0.03836537])

Para aquellos casos en los que los datos tengan muchos valores extremos, es posible que estandarizar usando la media y la desviacion estandar no funcione bien en el modelo. Para esos casos es mejor usar unos estimadores mas robustos (menos sensibles a outliers) y emplear un RobustScaler que funciona substrayendo la mediana y escalando mediante el rango intercuartil (IQR).

In [ ]:
escalador_robusto = preprocessing.RobustScaler()
var_numericas_imputadas_escalado_robusto = escalador_robusto.fit_transform(var_numericas_imputadas)

In [ ]:
print(var_numericas_imputadas_escalado_robusto.mean(axis=0))
print(var_numericas_imputadas_escalado_robusto.std(axis=0))

[-3.81916720e-17 -2.85106383e-02  4.01958704e-01  3.04018692e-02
  7.03130782e+01]
[6.33218559e-01 6.01245275e-01 1.38651621e+00 7.29970260e-01
 1.83690817e+03]


We say robus analisys if we have to do analys in wich the outliers don't affect as usually these made (It only check the data in the Inter quantil range)

**Escalado a un rango especifico**

Hay casos en los que en vez de estandardizar queremos escalar los datos a un rango (generalmente [-1,1] o [0,1]). Para ello podemos usar MinMaxScaler que hace escalado minmax (obviamente) o MaxAbscaler que simplemente divide cada valor de una variable por su valor máximo (y por tanto convierte el valor maximo a 1).

In [ ]:
print(var_numericas_imputadas.min(axis=0))
print(var_numericas_imputadas.max(axis=0))


[ 0.00000000e+00  0.00000000e+00  4.80888450e-02 -1.00000000e+02
 -2.07277465e+01]
[9.90000000e+01 9.90000000e+01 2.57710477e+01 8.91000000e+02
 1.07357858e+05]


In [ ]:
escalador_minmax = preprocessing.MinMaxScaler()
var_numericas_imputadas_escalado_minmax  = escalador_minmax.fit_transform(var_numericas_imputadas)

In [ ]:
print(var_numericas_imputadas_escalado_minmax.min(axis=0))
print(var_numericas_imputadas_escalado_minmax.max(axis=0)) # generate the image of each value in the columns image = (x-min)/(max-min) [0,1]
# Something interesting is that the outliers still being outliers (too close to 1 or 0) but more softs

[0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1.]


In [ ]:
escalador_maxabs = preprocessing.MaxAbsScaler()
var_numericas_imputadas_escalado_maxabs = escalador_maxabs.fit_transform(var_numericas_imputadas) # dive between the max of each columns

In [ ]:
print(var_numericas_imputadas_escalado_maxabs.min(axis=0))
print(var_numericas_imputadas_escalado_maxabs.max(axis=0))

[ 0.          0.          0.001866   -0.11223345 -0.00019307]
[1. 1. 1. 1. 1.]


Hay casos en los que lo que se necesita es tener observaciones con norma unitaria (norma L2 o euclidiana). Para esos casos, podemos usar Normalizer

In [ ]:
escalador_normalizador = preprocessing.Normalizer()
var_numericas_imputadas_escalado_normalizador = escalador_normalizador.fit_transform(var_numericas_imputadas)

In [ ]:
var_numericas_imputadas_escalado_normalizador[0] # x_11^2+x_12^2+x_13^2+...+x_1n^2 = 1 where n = no.columns

array([ 0.63288908,  0.55780055,  0.02395144, -0.53634668,  0.00827761])

**Variables Categoricas**

Los modelos están diseñados para trabajar con variables numéricas. Esto implica que para poder entrenar los modelos con variables categóricas tenemos que convertirlas a números. Este proceso se llama codificación (encoding)

In [ ]:
var_categoricas=datos[['col_categorica', 'col_ordinal']]

Hay muchas formas de codificar variables, la más sencilla es reemplazar los elementos de dichas variables por un número. Por ejemplo, si hacemos esto con la columna col_ordinal:

In [ ]:
var_categoricas['col_ordinal'].unique()

array(['muy bien', 'regular', 'muy mal', 'mal', 'bien'], dtype=object)

In [ ]:
label_codificador = preprocessing.LabelEncoder()
label_codificador.fit_transform(var_categoricas['col_ordinal'])[0:10]

array([2, 4, 3, 1, 0, 4, 0, 4, 0, 2])

In [ ]:

print(var_categoricas['col_ordinal'].values[0:10])
print(label_codificador.classes_)

['muy bien' 'regular' 'muy mal' 'mal' 'bien' 'regular' 'bien' 'regular'
 'bien' 'muy bien']
['bien' 'mal' 'muy bien' 'muy mal' 'regular']


  En el caso de variables ordinales esto tiene sentido ya que muy_bien>bien>regular>mal>muy mal. Sin embargo, esto indica a los modelos de scikit-learn por ejemplo que mal + regular = bien. Esto se puede usar en según que casos (hay modelos que no interpretan las variables numéricas así), o para codificar las variables objetivo.

Para variables categóricas (por ejemplo animales) no tiene sentido usar este tipo de encoding.

In [ ]:
label_codificador_categorico = preprocessing.LabelEncoder()
label_codificador_categorico.fit_transform(var_categoricas['col_categorica'])[0:10]

array([3, 0, 3, 1, 1, 2, 2, 2, 0, 0, 2, 0, 1, 3, 1, 3, 0, 3, 3, 0, 3, 3,
       3, 1, 1, 3, 1, 0, 1, 0, 0, 0, 0, 3, 0, 1, 0, 1, 1, 1, 1, 3, 0, 1,
       1, 2, 0, 1, 0, 2, 3, 2, 0, 3, 3, 1, 1, 0, 3, 2, 1, 1, 1, 1, 3, 0,
       0, 2, 1, 3, 2, 0, 1, 1, 3, 3, 0, 2, 3, 3, 0, 1, 2, 3, 1, 0, 0, 2,
       2, 2, 3, 1, 3, 1, 1, 0, 1, 1, 2, 2, 3, 2, 1, 0, 3, 2, 1, 1, 3, 1,
       2, 2, 2, 1, 2, 2, 0, 2, 3, 1, 1, 1, 1, 3, 3, 3, 2, 0, 0, 2, 3, 1,
       0, 1, 3, 3, 0, 1, 2, 3, 1, 0, 0, 0, 0, 2, 0, 0, 1, 3, 2, 2, 1, 2,
       2, 0, 1, 2, 2, 2, 2, 1, 3, 1, 1, 1, 2, 3, 3, 0, 3, 3, 0, 0, 3, 0,
       0, 0, 3, 3, 1, 2, 0, 0, 1, 1, 2, 0, 0, 0, 3, 2, 2, 1, 3, 1, 2, 3,
       0, 3, 3, 1, 3, 1, 3, 0, 1, 1, 2, 0, 2, 2, 1, 0, 0, 0, 1, 2, 1, 2,
       0, 0, 0, 0, 2, 1, 1, 0, 3, 2, 2, 3, 3, 2, 2, 0, 0, 2, 1, 2, 3, 2,
       3, 1, 3, 2, 0, 1, 1, 3, 3, 2, 1, 3, 3, 2, 0, 1, 3, 2, 0, 3, 0, 2,
       3, 2, 2, 2, 3, 3, 3, 2, 0, 2, 0, 1, 0, 0, 0, 0, 3, 0, 3, 0, 2, 3,
       1, 0, 2, 2, 3, 2, 3, 1, 1, 2, 1, 3, 2, 0, 1,

In [ ]:
label_codificador_categorico.classes_

array(['elefante', 'gato', 'perro', 'ratón'], dtype=object)

Digamos que no tiene sentido decir que la media de elefante y perro no es gato

Para estos casos una técnica que se puede usar se llama one-hot encoding. Lo que significa es que creamos n columnas binarias, con el valor 0 por defecto salvo la columna referente a la observación.

Para esto podemos usar OneHotEncoder

In [ ]:
oh_codificador = preprocessing.OneHotEncoder()


In [ ]:
oh_codificador.fit(datos.col_categorica)

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

Vemos que OneHotEncoder falla cuando se le pasan strings en vez de numeros. Por ello primero tenemos que convertir las variables categóricas a numéricas usando LabelEncoder.

In [ ]:
categorias_oh_codficadas = label_codificador_categorico.transform(datos.col_categorica)


In [ ]:
categorias_oh_codficadas

array([3, 0, 3, 1, 1, 2, 2, 2, 0, 0, 2, 0, 1, 3, 1, 3, 0, 3, 3, 0, 3, 3,
       3, 1, 1, 3, 1, 0, 1, 0, 0, 0, 0, 3, 0, 1, 0, 1, 1, 1, 1, 3, 0, 1,
       1, 2, 0, 1, 0, 2, 3, 2, 0, 3, 3, 1, 1, 0, 3, 2, 1, 1, 1, 1, 3, 0,
       0, 2, 1, 3, 2, 0, 1, 1, 3, 3, 0, 2, 3, 3, 0, 1, 2, 3, 1, 0, 0, 2,
       2, 2, 3, 1, 3, 1, 1, 0, 1, 1, 2, 2, 3, 2, 1, 0, 3, 2, 1, 1, 3, 1,
       2, 2, 2, 1, 2, 2, 0, 2, 3, 1, 1, 1, 1, 3, 3, 3, 2, 0, 0, 2, 3, 1,
       0, 1, 3, 3, 0, 1, 2, 3, 1, 0, 0, 0, 0, 2, 0, 0, 1, 3, 2, 2, 1, 2,
       2, 0, 1, 2, 2, 2, 2, 1, 3, 1, 1, 1, 2, 3, 3, 0, 3, 3, 0, 0, 3, 0,
       0, 0, 3, 3, 1, 2, 0, 0, 1, 1, 2, 0, 0, 0, 3, 2, 2, 1, 3, 1, 2, 3,
       0, 3, 3, 1, 3, 1, 3, 0, 1, 1, 2, 0, 2, 2, 1, 0, 0, 0, 1, 2, 1, 2,
       0, 0, 0, 0, 2, 1, 1, 0, 3, 2, 2, 3, 3, 2, 2, 0, 0, 2, 1, 2, 3, 2,
       3, 1, 3, 2, 0, 1, 1, 3, 3, 2, 1, 3, 3, 2, 0, 1, 3, 2, 0, 3, 0, 2,
       3, 2, 2, 2, 3, 3, 3, 2, 0, 2, 0, 1, 0, 0, 0, 0, 3, 0, 3, 0, 2, 3,
       1, 0, 2, 2, 3, 2, 3, 1, 1, 2, 1, 3, 2, 0, 1,

In [ ]:
categorias_oh_codficadas.shape

(1000,)

In [ ]:
categorias_oh_codficadas = oh_codificador.fit_transform(categorias_oh_codficadas.reshape(1000,1))

In [ ]:
categorias_oh_codficadas

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1000 stored elements and shape (1000, 4)>

Vemos que por defecto OneHotEncoder no devuelve un numpy array, sino una matriz sparse. La traducción sería "matriz escasa", y es una manera de representar matrices con muchos ceros (como es el caso de OneHot encoding) para consumir poca memoria.

Podemos convertir dichas matrices a arrays facilmente:

In [ ]:
categorias_oh_codficadas.toarray()

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

Vemos ahora la comparación en memoria de una matriz sparse versus su correspondiente np.array usando la función sys.getsizeof que devuelve el uso de memoria un objeto de python en bytes.

In [ ]:
import sys
sys.getsizeof(categorias_oh_codficadas)

56

In [ ]:
sys.getsizeof(categorias_oh_codficadas.toarray())

32128

Si queremos que el encoder devuelva arrays no tenemos más que pasarle el parametro sparse=False.

In [ ]:
oh_codificador = preprocessing.OneHotEncoder(sparse=False)

categorias_oh_codificadas = oh_codificador.fit_transform(categorias_codificadas.reshape(1000,1))
categorias_oh_codificadas

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

Pandas tiene la funcion auxiliar get_dummies que hace esto automáticamente de forma más fácil.

In [ ]:
np.array(pd.get_dummies(datos.col_categorica,dtype=int).head())

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0]])

#**Texto**


In [ ]:
from sklearn import feature_extraction

In [ ]:
datos.col_texto.values[:10]

array(['Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera.',
       'El resto della concluían sayo de velarte, calzas de velludo para las fiestas con sus pantuflos de lo mismo, los días de entre semana se honraba con su vellori de lo más fino.',
       'El resto della concluían sayo de velarte, calzas de velludo para las fiestas con sus pantuflos de lo mismo, los días de entre semana se honraba con su vellori de lo más fino.',
       'Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lentejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda.',
       'Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera.',
       'En un lugar de la Mancha, de cuyo nom

Para convertir texto en variables numéricas, podemos proceder de igual forma que con las variables categóricas, simplemente separando las palabras antes.

Para ello tenemos dos vectorizadores en scikit-learn, que convierten texto en vectores.

CountVectorizer devuelve un vector con el valor 0 en todas las palabras que no existen en una frase y con el numero de ocurrencias de las palabras que si existen

Vamos a hacer un ejemplo para que se vea bien.

In [ ]:
ejemplo_frases = ['los coches rojos',
          'los aviones son rojos',
          'los coches y los aviones son rojos',
          'los camiones rojos'
                 ]


vectorizador_count = feature_extraction.text.CountVectorizer()
X = vectorizador_count.fit_transform(ejemplo_frases)
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 15 stored elements and shape (4, 6)>

In [ ]:
vectorizador_count.get_feature_names_out()

array(['aviones', 'camiones', 'coches', 'los', 'rojos', 'son'],
      dtype=object)

In [ ]:
pd.DataFrame(X.toarray(), columns=vectorizador_count.get_feature_names_out())

,aviones,camiones,coches,los,rojos,son
0,0,0,1,1,1,0
1,1,0,0,1,1,1
2,1,0,1,2,1,1
3,0,1,0,1,1,0


El tomar simplemente el número de veces que aparece cada palabra tiene un problema, y es que da un mayor peso a aquellas palabras que aparecen muchas veces pero que no aportan ningun valor semántico (por ejemplo, los). Una manera más sofisticada de vectorizar un texto es en vez de usar el número de apariciones, usar TF-IDF. TF-IDF se traduce como Frecuencia de Texto - Frecuencia Inversa de Documento, y es una medida que asigna pesos a cada palabra en función de su frecuencia de aparición en todos los documents.


In [ ]:
vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
X = vectorizador_tfidf.fit_transform(ejemplo_frases)
pd.DataFrame(X.toarray(), columns=vectorizador_tfidf.get_feature_names_out())

,aviones,camiones,coches,los,rojos,son
0,0.000000,0.000000,0.730064,0.483222,0.483222,0.000000
1,0.589645,0.000000,0.000000,0.390280,0.390280,0.589645
2,0.438931,0.000000,0.438931,0.581047,0.290524,0.438931
3,0.000000,0.804612,0.000000,0.419880,0.419880,0.000000


In [ ]:
vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
texto_vectorizado = vectorizador_tfidf.fit_transform(datos.col_texto)
texto_vectorizado

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 28295 stored elements and shape (1000, 134)>

In [ ]:
texto_vectorizado = texto_vectorizado.toarray()
texto_vectorizado

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.19788701, 0.19788701, 0.        , ..., 0.        , 0.        ,
        0.19788701],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
label_codificador.classes_

array(['bien', 'mal', 'muy bien', 'muy mal', 'regular'], dtype=object)

**Poniendolo todo junto**

In [ ]:
col_numericas =  ['col_inexistente1', 'col2', 'col3', 'col_outliers', 'col_outliers2']
col_categorica = ['col_categorica']
col_texto = ['col_texto']


#Variables numéricas
imputador = SimpleImputer(missing_values=np.nan, copy=False, strategy="mean")
escalador = preprocessing.StandardScaler()
var_numericas_imputadas_escalado_standard = escalador.fit_transform(
                                                imputador.fit_transform(datos[col_numericas])
                                            )
df_numerico_procesado = pd.DataFrame(var_numericas_imputadas_escalado_standard,
                                                   columns=col_numericas)


# Variable categorica (dumies)
label_codificador_categorico = preprocessing.LabelEncoder()
categorias_codificadas = label_codificador_categorico.fit_transform(datos[col_categorica])
oh_codificador = preprocessing.OneHotEncoder(sparse_output=False)
categorias_oh_codificadas = oh_codificador.fit_transform(categorias_codificadas.reshape(1000,1))

df_categorico_procesado = pd.DataFrame(categorias_oh_codificadas,
                                       columns=label_codificador_categorico.classes_)


# Texto (mineria phrases, could be useful apply stop words in R)-
vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
texto_vectorizado = vectorizador_tfidf.fit_transform(datos.col_texto)
df_texto_procesado =  pd.DataFrame(texto_vectorizado.toarray(), columns=vectorizador_tfidf.get_feature_names_out())

#union of columns by row
datos_procesados = pd.concat([
    df_numerico_procesado,
    df_categorico_procesado,
    df_texto_procesado
], axis=1)

# variable ordinal (hierarchy)
label_codificador_ordinal = preprocessing.LabelEncoder()
datos_procesados['col_ordinal'] = label_codificador_ordinal.fit_transform(datos.col_ordinal)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,col_inexistente1,col2,col3,col_outliers,col_outliers2,elefante,gato,perro,ratón,acordarme,...,vaca,veinte,velarte,vellori,velludo,verdad,verosímiles,viernes,vivía,col_ordinal
0,0.399217,0.082807,0.442819,-0.694600,-0.038365,0.0,0.0,0.0,1.0,0.000000,...,0.000000,0.204745,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,2
1,-0.653605,0.861333,-0.323390,-0.118466,-0.038278,1.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.181842,0.181842,0.181842,0.0,0.0,0.000000,0.000000,4
2,1.226435,-0.766494,-0.484752,-0.464146,-0.038343,0.0,0.0,0.0,1.0,0.000000,...,0.000000,0.000000,0.181842,0.181842,0.181842,0.0,0.0,0.000000,0.000000,3
3,-0.540803,-1.191145,-0.375028,-1.129901,-0.038405,0.0,1.0,0.0,0.0,0.000000,...,0.194272,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.194272,0.000000,1
4,-0.616004,-0.766494,-0.516874,0.777743,-0.037257,0.0,1.0,0.0,0.0,0.000000,...,0.000000,0.204745,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.376838,0.365907,0.492254,0.611304,-0.038492,1.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,3
996,-0.352799,-1.615795,0.131503,0.700925,-0.037878,0.0,0.0,0.0,1.0,0.000000,...,0.000000,0.204745,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,2
997,-1.104815,0.118194,-0.420944,0.880166,-0.038197,1.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.181842,0.181842,0.181842,0.0,0.0,0.000000,0.000000,4
998,1.489641,1.569084,-0.407745,0.816152,-0.038190,1.0,0.0,0.0,0.0,0.197887,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.197887,0


In [ ]:
datos_procesados


,col_inexistente1,col2,col3,col_outliers,col_outliers2,elefante,gato,perro,ratón,acordarme,...,vaca,veinte,velarte,vellori,velludo,verdad,verosímiles,viernes,vivía,col_ordinal
0,0.399217,0.082807,0.442819,-0.694600,-0.038365,0.0,0.0,0.0,1.0,0.000000,...,0.000000,0.204745,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,2
1,-0.653605,0.861333,-0.323390,-0.118466,-0.038278,1.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.181842,0.181842,0.181842,0.0,0.0,0.000000,0.000000,4
2,1.226435,-0.766494,-0.484752,-0.464146,-0.038343,0.0,0.0,0.0,1.0,0.000000,...,0.000000,0.000000,0.181842,0.181842,0.181842,0.0,0.0,0.000000,0.000000,3
3,-0.540803,-1.191145,-0.375028,-1.129901,-0.038405,0.0,1.0,0.0,0.0,0.000000,...,0.194272,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.194272,0.000000,1
4,-0.616004,-0.766494,-0.516874,0.777743,-0.037257,0.0,1.0,0.0,0.0,0.000000,...,0.000000,0.204745,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.376838,0.365907,0.492254,0.611304,-0.038492,1.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,3
996,-0.352799,-1.615795,0.131503,0.700925,-0.037878,0.0,0.0,0.0,1.0,0.000000,...,0.000000,0.204745,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,2
997,-1.104815,0.118194,-0.420944,0.880166,-0.038197,1.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.181842,0.181842,0.181842,0.0,0.0,0.000000,0.000000,4
998,1.489641,1.569084,-0.407745,0.816152,-0.038190,1.0,0.0,0.0,0.0,0.197887,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.197887,0
